In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 发送请求获取网页内容
url = "https://worldperatio.com/sp-500-sectors/"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# 查找包含表格的部分
table = soup.find('table', class_="money home-all sortable w3-table -f14 pd44")

# 解析表格头部
headers = []
for th in table.find_all('th'):
    # print(th)
    headers.append(th.get_text(strip=True))

# 解析表格行
rows = []
for tr in table.find_all('tr')[1:]:  # 跳过表头
    cols = tr.find_all('td')
    if len(cols) > 0:
        row = [col.get_text(strip=True) for col in cols]
        rows.append(row)

df = pd.DataFrame(rows, columns=headers[9:])

In [44]:
df

,,S&P 500 Sector,,Trend,P/E▾,5 Years,10 Years,20 Years,5 Years,10 Years,20 Years,5 Years,10 Years,20 Years,
0,,Real Estate,XLRE,,37.27,Fair,Fair,-,39.18,38.06,-,-0.31 σ,-0.21 σ,-,07 Feb
1,,Information Technology,XLK,,37.05,Expensive,Expensive,Expensive,28.85,22.59,18.33,+2.38 σ,+2.70 σ,+4.34 σ,07 Feb
2,,Health Care,XLV,,31.82,Expensive,Expensive,Expensive,21.81,19.92,17.43,+2.16 σ,+5.89 σ,+5.97 σ,07 Feb
3,,Consumer Discretionary,XLY,,29.06,Fair,Overvalued,Expensive,27.78,23.11,19.21,+0.41 σ,+1.54 σ,+2.81 σ,07 Feb
4,,S&P 500 Index,SPY,,27.00,Expensive,Expensive,Expensive,21.34,18.56,15.89,+2.47 σ,+3.66 σ,+4.78 σ,07 Feb
5,,Industrials,XLI,,26.34,Expensive,Expensive,Expensive,21.72,18.68,16.40,+2.01 σ,+2.81 σ,+4.64 σ,07 Feb
6,,Materials,XLB,,24.88,Expensive,Expensive,Expensive,18.81,17.28,15.59,+2.20 σ,+4.22 σ,+4.92 σ,07 Feb
7,,Consumer Staples,XLP,,22.62,Fair,Overvalued,Expensive,21.78,20.29,17.94,+0.51 σ,+1.67 σ,+2.28 σ,07 Feb
8,,Communication Services,XLC,,20.89,Fair,Overvalued,Expensive,20.91,17.21,15.65,-0.01 σ,+1.03 σ,+2.09 σ,07 Feb
9,,Utilities,XLU,,20.59,Overvalued,Overvalued,Expensive,19.49,18.25,16.23,+1.08 σ,+1.86 σ,+2.30 σ,07 Feb


In [41]:
# print(rows[0])

In [42]:
# print(headers[9:])

In [45]:
index_output_dict = {
        'https://worldperatio.com/index/sp-500/'	: 'sp-500',
        'https://worldperatio.com/sector/sp-500-utilities/'	: 'sp-500-utilities',
        'https://worldperatio.com/sector/sp-500-real-estate/'	: 'sp-500-real-estate',
        'https://worldperatio.com/sector/sp-500-materials/'	: 'sp-500-materials',
        'https://worldperatio.com/sector/sp-500-information-technology/'	: 'sp-500-information-technology',
        'https://worldperatio.com/sector/sp-500-industrials/'	: 'sp-500-industrials',
        'https://worldperatio.com/sector/sp-500-healthcare/'	: 'sp-500-healthcare',
        'https://worldperatio.com/sector/sp-500-financials/'	: 'sp-500-financials',
        'https://worldperatio.com/sector/energy/'	: 'energy',
        'https://worldperatio.com/sector/sp-500-consumer-staples/'	: 'sp-500-consumer-staples',
        'https://worldperatio.com/sector/sp-500-consumer-discretionary/'	: 'sp-500-consumer-discretionary',
        'https://worldperatio.com/sector/sp-500-communication-services/'	: 'sp-500-communication-services'

    }


In [125]:
def check_string(s):
    # 定义需要匹配的规则
    rule1 = r'full_isin_chart_data\["[A-Z]+"\] ='
    rule2 = r'full_isin_chart_data_(avg|range|2_range)\["[A-Z]+"\]\[\d+\] ='

    # 检查是否包含任意一个关键字
    if "full_isin_chart_data" in s:
        if "full_isin_chart_data_avg" in s or "full_isin_chart_data_range" in s or "full_isin_chart_data_2_range" in s:
            # 满足条件2、3或4，检查是否符合规则2
            if re.search(rule2, s):
                return True, 'rule1'
        else:
            # 满足条件1，检查是否符合规则1
            if re.search(rule1, s):
                return True, 'rule2'
    return False, ''

In [164]:
def parse_string(s):
    name, value = s.split('=')
    name_tm = ''
    if len(name.split('[')) == 3:
        name, name_tk, name_tm = name.split('[')
    else:
        name, name_tk = name.split('[')
    name_tk = name_tk.strip('\[\]" \n')
    name_tm = name_tm.strip('\[\]" \n')
    name = name.strip('\[\]" \n')

    # print(name, name_tk, name_tm, value[0:100])
    return name, name_tk, name_tm, value

In [149]:
def parse_date_utc(match):
    # 提取年月日，并转换为日期
    date_str = match.group(1)
    # 使用 datetime.strptime() 来解析日期
    print(date_str)
    date_obj = datetime.strptime(date_str, '%Y, %m, %d')
    # 返回时间戳
    return str(int(date_obj.timestamp() * 1000))  # 返回毫秒时间戳

# 1. 替换 Date.UTC() 为时间戳
def parse_date_utc(match):
    # 提取年月日
    year = int(match.group(1))
    month = int(match.group(2)) + 1  # JavaScript 月份从 0 开始，Python 从 1 开始
    day = int(match.group(3))

    # 使用 datetime 构造日期对象
    date_obj = datetime(year, month, day)

    # 返回时间戳（秒为单位）
    return str(int(date_obj.timestamp() * 1000))  # 返回毫秒时间戳


In [175]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime
import ast

# 发送请求获取网页内容
url = "https://worldperatio.com/sector/sp-500-real-estate/"
url = "https://worldperatio.com/sector/sp-500-industrials/"
url = 'https://worldperatio.com/index/sp-500/'
url = 'https://worldperatio.com/sector/sp-500-materials/'

response = requests.get(url)

# text = response.text
soup = BeautifulSoup(response.text, 'html.parser')

# 查找指定 id 的 div 元素
# div_element = soup.find('div', id='full_isin_chart').text
scripts = soup.find_all('script', type='text/javascript')
script = [script for script in scripts if 'full_isin_chart_data' in scripts]

result = {}
for s in scripts:
    text = s.get_text()
    
    if not 'full_isin_chart_data' in text:
        continue

    fields = text.split(';')
    for idx, field in enumerate(fields):
        flag, rule = check_string(field)
        if flag:
            name, name_ti, name_tm, value = parse_string(field)
            # print('process name:', name, '|    name_ti:', name_ti, '|    name_tm:', name_tm)
            
            # 正则替换 Date.UTC() 语法为时间戳
            updated_string = re.sub(r'Date.UTC\((\d{4}), (\d+), (\d+)\)', parse_date_utc, value)
            # print(name, updated_string[0:60])
            # 2. 将字符串转换为 Python 列表
            data = ast.literal_eval(updated_string)
            print(name, data[0:3])
            
            # 3. 转换日期为 datetime 对象，并构建 Pandas DataFrame
            parsed_data = [(datetime.utcfromtimestamp(item[0] / 1000), item[1:]) for item in data]
            print('============================')
            print(parsed_data)
            
            # 创建 DataFrame
            # if not '_range' in name:
            df = pd.DataFrame(parsed_data, columns=["Date", "Value"])
            # else:
            #     print(parsed_data)
            #     df = pd.DataFrame(parsed_data, columns=["Date", "upper", "lower"])

            key = '_'.join([name, name_ti, name_tm])
            result[key] = df
            
            # 打印 DataFrame
            print(df)

full_isin_chart_data [[788889600000, 12.1194], [791568000000, 11.7042], [793987200000, 10.949]]
[(datetime.datetime(1994, 12, 31, 16, 0), [12.1194]), (datetime.datetime(1995, 1, 31, 16, 0), [11.7042]), (datetime.datetime(1995, 2, 28, 16, 0), [10.949]), (datetime.datetime(1995, 3, 31, 16, 0), [10.949]), (datetime.datetime(1995, 4, 30, 16, 0), [10.5248]), (datetime.datetime(1995, 5, 31, 16, 0), [10.595]), (datetime.datetime(1995, 6, 30, 16, 0), [10.595]), (datetime.datetime(1995, 7, 31, 16, 0), [10.6906]), (datetime.datetime(1995, 8, 31, 16, 0), [10.6906]), (datetime.datetime(1995, 9, 30, 16, 0), [9.955]), (datetime.datetime(1995, 10, 31, 16, 0), [9.955]), (datetime.datetime(1995, 11, 30, 16, 0), [9.955]), (datetime.datetime(1995, 12, 31, 16, 0), [10.7632]), (datetime.datetime(1996, 1, 31, 16, 0), [12.379]), (datetime.datetime(1996, 2, 29, 16, 0), [13.385]), (datetime.datetime(1996, 3, 31, 16, 0), [14.2652]), (datetime.datetime(1996, 4, 30, 16, 0), [14.8201]), (datetime.datetime(1996, 5,

In [155]:
for key, df in result.items():
    print(key, '=============')
    print(df)

full_isin_chart_data_XLI_ =============
                   Date    Value
0   1995-01-31 16:00:00  12.6702
1   1995-02-28 16:00:00  12.6702
2   1995-03-31 16:00:00  12.8901
3   1995-04-30 16:00:00  13.3307
4   1995-05-31 16:00:00  13.3307
..                  ...      ...
356 2024-09-30 16:00:00  27.3467
357 2024-10-31 16:00:00  25.9829
358 2024-11-30 16:00:00  26.3367
359 2024-12-31 16:00:00  26.1971
360 2025-01-31 16:00:00  26.5759

[361 rows x 2 columns]
full_isin_chart_data_avg_XLI_12 =============
                   Date      Value
0   1996-01-31 16:00:00  13.346089
1   1996-02-29 16:00:00  13.466344
2   1996-03-31 16:00:00  13.636200
3   1996-04-30 16:00:00  13.757100
4   1996-05-31 16:00:00  13.937089
..                  ...        ...
344 2024-09-30 16:00:00  23.623167
345 2024-10-31 16:00:00  24.501922
346 2024-11-30 16:00:00  25.023422
347 2024-12-31 16:00:00  25.469256
348 2025-01-31 16:00:00  25.734211

[349 rows x 2 columns]
full_isin_chart_data_avg_XLI_60 =============
    

[[1136044800000,35.7584],[1138723200000,40.9824],[1141142400000,39.3696],[1143820800000,37.3147],[1146412800000,35.8828],[1149091200000,36.0511],[1151683200000,30.3267],[1154361600000,32.8160],[1157040000000,35.6309],[1159632000000,41.5667],[1162310400000,40.7987],[1164902400000,39.7977],[1167580800000,42.4143],[1170259200000,37.1261],[1172678400000,33.9225],[1175356800000,34.5966],[1177948800000,35.9165],[1180627200000,34.0792],[1183219200000,34.1798],[1185897600000,28.8915],[1188576000000,31.1914],[1191168000000,34.4544],[1193846400000,29.1837],[1196438400000,27.9405],[1199116800000,24.8874],[1201795200000,27.5224],[1204300800000,28.4064],[1206979200000,30.0017],[1209571200000,29.8801],[1212249600000,29.1170],[1214841600000,25.0192],[1217520000000,29.2825],[1220198400000,30.3738],[1222790400000,21.9519],[1225468800000,21.6512],[1228060800000,28.4200],[1230739200000,26.7509],[1233417600000,22.1683],[1235836800000,27.9245],[1238515200000,34.9047],[1241107200000,41.9578],[1243785600000,